In [3]:
!pip install fpdf


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40769 sha256=292d71beba05b246859045242fd4038571ff1e60ca1a1c6d3d558694999c2dac
  Stored in directory: c:\users\ocs\appdata\local\pip\cache\wheels\aa\da\11\a3189f34ddc13c26a2d0f329eac46b728c7f31c39e4dc26243
Successfully built fpdf


  DEPRECATION: Building 'fpdf' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'fpdf'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from fpdf import FPDF
from tqdm import tqdm

# --- STEP 1: Load dataset ---
file_path = "psx_data_20211210_to_20241210.csv"
df = pd.read_csv(file_path)

# --- STEP 2: Clean dataset ---
df = df.drop_duplicates()
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.dropna(subset=['Date', 'Symbol', 'Open', 'High', 'Low', 'Close'])
df = df.sort_values(by=['Symbol', 'Date']).reset_index(drop=True)

# --- STEP 3: Create output folder ---
output_folder = "psx_charts"
os.makedirs(output_folder, exist_ok=True)

# --- STEP 4: Generate chart for each company ---
symbols = df['Symbol'].unique()
print(f"📈 Generating {len(symbols)} stock charts...")

plt.style.use('seaborn-v0_8-darkgrid')

for symbol in tqdm(symbols):
    symbol_data = df[df['Symbol'] == symbol]
    if len(symbol_data) < 2:
        continue

    plt.figure(figsize=(10, 5))
    plt.plot(symbol_data['Date'], symbol_data['Close'], color='steelblue', linewidth=1.5)
    plt.title(f'{symbol} - Daily Closing Price Trend', fontsize=14)
    plt.xlabel('Date')
    plt.ylabel('Closing Price (PKR)')
    plt.tight_layout()
    
    chart_path = os.path.join(output_folder, f"{symbol}_trend.png")
    plt.savefig(chart_path, dpi=200)
    plt.close()

print("\n✅ Individual stock charts saved successfully!")

# --- STEP 5: Create summary charts ---
# Top 10 average prices
avg_prices = df.groupby('Symbol')['Close'].mean().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 5))
avg_prices.plot(kind='bar', color='green')
plt.title('Top 10 Companies by Average Closing Price')
plt.xlabel('Symbol')
plt.ylabel('Average Close (PKR)')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "Top10_AvgPrice.png"), dpi=200)
plt.close()

# Top 10 volatile stocks
df['Volatility'] = df['High'] - df['Low']
avg_volatility = df.groupby('Symbol')['Volatility'].mean().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 5))
avg_volatility.plot(kind='bar', color='orange')
plt.title('Top 10 Most Volatile Stocks')
plt.xlabel('Symbol')
plt.ylabel('Average Daily Range (PKR)')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "Top10_Volatile.png"), dpi=200)
plt.close()

print("📊 Summary charts created successfully!")

# --- STEP 6: Generate PDF Report ---
print("\n🧾 Creating PDF report...")

pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Cover page
pdf.add_page()
pdf.set_font("Arial", "B", 20)
pdf.cell(0, 15, "Pakistan Stock Exchange Report", ln=True, align="C")
pdf.set_font("Arial", "", 14)
pdf.cell(0, 10, "Date Range: {} to {}".format(df['Date'].min().date(), df['Date'].max().date()), ln=True, align="C")
pdf.ln(10)
pdf.set_font("Arial", "", 12)
pdf.multi_cell(0, 8, "This report contains daily stock price trends and summary analysis for all listed companies "
                     "in the PSX dataset. Each page shows the historical closing price trend of a company, "
                     "along with overall summary visuals at the end.")
pdf.ln(10)

# Add each chart
chart_files = sorted([f for f in os.listdir(output_folder) if f.endswith('.png')])

for chart in tqdm(chart_files):
    pdf.add_page()
    pdf.set_font("Arial", "B", 14)
    pdf.cell(0, 10, chart.replace("_trend.png", "").replace(".png", ""), ln=True, align="C")
    pdf.image(os.path.join(output_folder, chart), x=10, y=30, w=190)
    pdf.ln(85)

# Save the PDF
pdf_path = "PSX_Report.pdf"
pdf.output(pdf_path)

print(f"\n✅ PDF Report generated successfully: {pdf_path}")


📈 Generating 90 stock charts...


100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:32<00:00,  2.76it/s]



✅ Individual stock charts saved successfully!
📊 Summary charts created successfully!

🧾 Creating PDF report...


100%|██████████████████████████████████████████████████████████████████████████████████| 92/92 [07:30<00:00,  4.90s/it]



✅ PDF Report generated successfully: PSX_Report.pdf
